<a href="https://colab.research.google.com/github/robgon-art/ai-tunes/blob/main/AI_Tunes_Generate_Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI-Tunes: Creating New Songs with Artificial Intelligence**
### **How I fine-tuned OpenAI's GPT-3 to generate music with a global structure**
By Robert. A Gonsalves

You can read my article about this project on Medium.

The source code is released under the CC BY-SA license.

![CC BY-NC-SA](https://licensebuttons.net/l/by-sa/4.0/88x31.png)


In [ ]:
#@title **Initalize the System**
#@markdown Hover over play button and hit the Run cell.</br>
#@markdown HIt takes about 3 minutes to complete the setup.
!git clone https://github.com/robgon-art/music-geometry-eval
!pip install openai

import sys
sys.path.append('/content/music-geometry-eval/music_geometry_eval')

import music_geometry_eval
import openai
import music21
import numpy as np
from collections.abc import Iterable

!gsutil -q -m cp -r gs://magentadata/models/music_transformer/primers/* /content/
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install magenta
!pip install pyfluidsynth
import note_seq
SF2_PATH = '/content/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000
CMM_mean = 2.2715
CMM_std = 0.4831
LM_mean = 2.0305
LM_std = 0.5386
CENT_mean = 0.3042
CENT_std = 0.0891

**Specify Your OpenAI API Key**</br>
This Colab only works if you have an account with OpenAI.</br>
If you don't have an account you can sign up here https://openai.com/blog/openai-api/

In [ ]:
openai.api_key = "<Your OpenAI API Key>"

In [ ]:
#@title **Generate a new Song Title and Band Name**
#@markdown This data will be used to prompt the AI-Tunes system to create the song.
response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt="Create a new song title a new band name. Be creative!\n\nBand name: The Execs\nSong title: Company Meeting\n###\n\nBand name: The One Chords\nSong title: Moving Down to Burlington\n###",
  temperature=0.7,
  max_tokens=64,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.5,
  stop=["###"]
)

# print(response)

song_metadata = response["choices"][0]["text"].strip()
lines = song_metadata.split("\n")
generated_metadata = {}
song_title = "Our Random Song"
band_name = "Some Random Band"
for line in lines:
  parts = line.split(":")
  if len(parts) == 2:
    if "song title" in parts[0].lower() and len(parts[1]) > 0:
      song_title = parts[1].strip()
    if "band name" in parts[0].lower() and len(parts[1]) > 0:
      band_name = parts[1].strip()

print("Song Title:", song_title)
print("Band Name :", band_name)

In [ ]:
#@title **Generate Songs**
#@markdown The sysystem will generate five versions of the song and show the statistics for tonal quality.</br>
#@markdown Lower temperature values will create more repetive melodies.</br>
#@markdown Higher values will create more random melodies (and occasional ABC parsing errors.)
temperature = 0.75 #@param {type:"slider", min:0.0, max:1.2, step:0.01}

prompt = "X: 1 $ T: " + song_title + " $ C: " + band_name + " $ <song>"
print("prompt", prompt)
print()

songs_with_scores = []
score_arr = np.empty((0), np.float32)

for i in range(5):
  print("\nGenerating Song Version", i)
  response = openai.Completion.create(
      model="curie:ft-user-j0julqovorjakyuyt3kv3zci-2021-08-24-11-42-59",
      prompt=prompt,
      stop = " $ <end>",
      temperature=temperature,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      max_tokens = 1000)

  # print(response)
  # print()

  formatted_prompt = "X: 1 $ T: " + song_title + " $ C: " + band_name + " $ L: 1/4 $ M: 4/4 $ K: C $ V: 1 treble"
  formatted_prompt = formatted_prompt.replace(" $ ", "\n")
  formatted_prompt = formatted_prompt.replace("<song>", "").strip()

  formatted_song = response["choices"][0]["text"].strip()
  formatted_song = formatted_song.replace('`', '"')
  formatted_song = formatted_song.replace(" $ ", "\n")
  new_song = formatted_prompt + "\n" + formatted_song
  # print(new_song)
  with open("new_song.abc", "w") as new_song_file:
    new_song_file.write(new_song)

  song = music21.converter.parse("new_song.abc")

  part = song.parts[0]
  chord_end = song.highestTime
  for pi in reversed(range(len(part))):
    p = part[pi]
    for ni in reversed(range(len(p))):
      n = p[ni]
      if type(n) == music21.harmony.ChordSymbol:
        chord_start = p.offset + n.offset
        n.duration.quarterLength = chord_end - chord_start
        n.volume = music21.volume.Volume(velocity=48)
        chord_end = chord_start
      elif type(n) == music21.note.Note:
        n.volume = music21.volume.Volume(velocity=64)
  file_name = "song" + str(i).zfill(2) + ".mid"
  song.write('midi', fp=file_name)

  part = song.parts[0]
  note_array = []

  for p in part:
    if isinstance(p, Iterable):
      for n in p:
        if type(n) == music21.note.Note:
          note_array.append([int(n.pitch.ps), int(n.quarterLength*4+0.5)])

  CMM = music_geometry_eval.calculate_time_supported_conjunct_melodic_motion(note_array)
  LM = music_geometry_eval.calculate_time_supported_limited_macroharmony(note_array, span_size=32)
  CENT = music_geometry_eval.calculate_time_supported_centricity(note_array, span_size=32)

  print("  CMM :", round(CMM, 4))
  print("  LM  :", round(LM, 4))
  print("  CENT:", round(CENT, 4))

  norm_cmm = (CMM - CMM_mean) / CMM_std
  norm_lm = (LM - LM_mean) / LM_std
  norm_cent = (CENT - CENT_mean) / CMM_std
  norm_score_squared = norm_cmm * norm_cmm + norm_lm * norm_lm + norm_cent * norm_cent
  print("  NDM:", round(norm_score_squared, 4))
  score_arr = np.append(score_arr, norm_score_squared)
  songs_with_scores.append([norm_score_squared, file_name])

print("\nResults:")
songs_with_scores.sort()
for i, pair in enumerate(songs_with_scores):
  print("Version: " + str(i) + ", Score: " + str(round(pair[0], 4)) + ", File: ", pair[1])

In [ ]:
#@title **Choose a Song**
#@markdown Choose a version of the song to play.

version_number = 0 #@param {type:"slider", min:0, max:5, step:1}
melody_ns = note_seq.midi_file_to_sequence_proto(songs_with_scores[version_number][1])
print("\nSong Title: " + song_title)
print("Version:   ", version_number)
print("Band Name: ", band_name)
print("NDM Score: ", round(songs_with_scores[version_number][0], 4))
print("MIDI File: ", songs_with_scores[version_number][1], "\n")

note_seq.play_sequence( 
  melody_ns,
  synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
# note_seq.plot_sequence(melody_ns)

from bokeh.plotting import show
fig = note_seq.plot_sequence(melody_ns, show_figure = False)
fig.width = 1500
fig.height = 500
fig.toolbar.logo = None
# fig.toolbar_location = None
show(fig)